### Installing packages

In [1]:
# Installing packages
#!pip3 install pandas
#!pip3 install biopython

# Importing packages
import Bio
import math
import os
import sys
import argparse
import shutil
import re
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.Data import CodonTable
from Bio.SeqRecord import SeqRecord
from Bio import Entrez
from itertools import chain
from tqdm import tqdm
#from google.colab import files   # For working with colab only
#from google.colab import drive   # For working with colab only

# Asserting email for Entrez
Entrez.email = "bogdan.sotnikov.1999@mail.ru"

# Google drive mounting
#drive.mount('/content/gdrive')    # For working with colab only

In [ ]:
parser = argparse.ArgumentParser()

parser.add_argument("path", type=str)
parser.add_argument("mail", type=str)
parser.add_argument("threads", type=int)

arguments = parser.parse_args()

json_file_name = arguments.path
Entrez.email = arguments.mail
threads = arguments.threads
# Если аргумент есть в аргументах, сделать api

In [ ]:
json_path = os.path.join("../data/jsons", json_file_name)
with open(json_path, "r") as json_organism:
    json_organism = json.load(json_organism)


organism_name = json_organism[0]
complete_ids = list(json_organism[1].values())[0]

In [20]:
if type(complete_ids[len(complete_ids)-1]) == list:
    scaffs = complete_ids[len(complete_ids)-1]
    del complete_ids[len(complete_ids)-1]
    complete_ids += scaffs

In [58]:
# Code from source above, with modifications. It is for choosing GenBank nucleotide data only.
def extract_insdc(links):
    linkset = [ls for ls in links[0]['LinkSetDb'] if
              ls['LinkName'] == 'assembly_nuccore_refseq']
    if 0 != len(linkset):
        uids = [link['Id'] for link in linkset[0]['Link']]
    else:
        uids = 0
    return uids

In [23]:
# Creating a variables for futher work with links
#organism = organism_name
organism = organism_name
db_search = "assembly"
db_current = "nucleotide"

In [6]:
# # Getting summary about links
# idlist = search_record["IdList"]
# complete_ids = []   # List of full completed genomes' ids
# for ids in tqdm(idlist):
#     handle = Entrez.esummary(db=db_search, id=ids)
#     record = Entrez.read(handle)
#     if record['DocumentSummarySet']['DocumentSummary'][0]['AssemblyStatus'] == "Complete Genome":
#         complete_ids.append(ids)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 203/203 [06:04<00:00,  1.80s/it]


In [59]:
# Taking ids for fetching. It collected all non-duplicated links in nucleotide database from assembly database.
links = []
links_checked = []
n = 0
for complete_id in tqdm(complete_ids):
    link_handle = Entrez.elink(dbfrom=db_search, db=db_current, from_uid=complete_id)
    link_record = Entrez.read(link_handle)
    uids = extract_insdc(link_record)
    if uids != 0:
        for uid in uids:
            print(uid)
            if uid not in links_checked:  # Checking for duplicates
                print("append")
                links_checked.append(uid)
                links.append((uid, n))
                cumulative = 1
            else:
                cumulative = 0
        n += cumulative

 17%|█████████████████▋                                                                                        | 1/6 [00:01<00:08,  1.61s/it]

[{'LinkSetDb': [{'Link': [{'Id': '2482828375'}, {'Id': '2482828374'}, {'Id': '2482828373'}, {'Id': '2482828372'}, {'Id': '2482828371'}, {'Id': '2482828369'}, {'Id': '2482433114'}, {'Id': '2482433099'}, {'Id': '2482433084'}, {'Id': '2482433057'}, {'Id': '2482430655'}, {'Id': '2482430642'}], 'DbTo': 'nuccore', 'LinkName': 'assembly_nuccore'}, {'Link': [{'Id': '2482433114'}, {'Id': '2482433099'}, {'Id': '2482433084'}, {'Id': '2482433057'}, {'Id': '2482430655'}, {'Id': '2482430642'}], 'DbTo': 'nuccore', 'LinkName': 'assembly_nuccore_insdc'}, {'Link': [{'Id': '2482828375'}, {'Id': '2482828374'}, {'Id': '2482828373'}, {'Id': '2482828372'}, {'Id': '2482828371'}, {'Id': '2482828369'}], 'DbTo': 'nuccore', 'LinkName': 'assembly_nuccore_refseq'}], 'ERROR': [], 'LinkSetDbHistory': [], 'DbFrom': 'assembly', 'IdList': ['16372881']}]
2482828375
append
2482828374
append
2482828373
append
2482828372
append
2482828371
append
2482828369
append


 33%|███████████████████████████████████▎                                                                      | 2/6 [00:03<00:06,  1.51s/it]

[{'LinkSetDb': [{'Link': [{'Id': '2475576196'}, {'Id': '2475576195'}, {'Id': '2475576194'}, {'Id': '2475576193'}, {'Id': '2474459373'}, {'Id': '2474459372'}, {'Id': '2474459371'}, {'Id': '2474459370'}], 'DbTo': 'nuccore', 'LinkName': 'assembly_nuccore'}, {'Link': [{'Id': '2474459373'}, {'Id': '2474459372'}, {'Id': '2474459371'}, {'Id': '2474459370'}], 'DbTo': 'nuccore', 'LinkName': 'assembly_nuccore_insdc'}, {'Link': [{'Id': '2475576196'}, {'Id': '2475576195'}, {'Id': '2475576194'}, {'Id': '2475576193'}], 'DbTo': 'nuccore', 'LinkName': 'assembly_nuccore_refseq'}], 'ERROR': [], 'LinkSetDbHistory': [], 'DbFrom': 'assembly', 'IdList': ['16357271']}]
2475576196
append
2475576195
append
2475576194
append
2475576193
append


 50%|█████████████████████████████████████████████████████                                                     | 3/6 [00:04<00:04,  1.61s/it]

...

[{'LinkSetDb': [{'Link': [{'Id': '2205696692'}, {'Id': '2205696691'}, {'Id': '2205696690'}, {'Id': '2205696689'}, {'Id': '2205696688'}, {'Id': '2205696686'}, {'Id': '2205696685'}, {'Id': '2205696684'}, {'Id': '2205696683'}, {'Id': '2205696682'}, {'Id': '2205696681'}, {'Id': '2205696680'}, {'Id': '2205696679'}, {'Id': '2205696678'}, {'Id': '2205696677'}, {'Id': '2205696675'}, {'Id': '2205696674'}, {'Id': '2205696673'}, {'Id': '2205696672'}, {'Id': '2205696667'}, {'Id': '2205696665'}, {'Id': '2205696660'}, {'Id': '2205696658'}, {'Id': '2205696657'}, {'Id': '2205696655'}, {'Id': '2205696649'}, {'Id': '2205696647'}, {'Id': '2205696645'}, {'Id': '2205696642'}, {'Id': '2205696640'}, {'Id': '2205696638'}, {'Id': '2205696637'}, {'Id': '2205696635'}, {'Id': '2205696630'}, {'Id': '2205696628'}, {'Id': '2205696627'}, {'Id': '2205696625'}, {'Id': '2205696623'}, {'Id': '2205696621'}, {'Id': '2205696617'}, {'Id': '2205696612'}, {'Id': '2205696611'}, {'Id': '2205696610'}, {'Id': '2205696609'}, {'Id':

 83%|████████████████████████████████████████████████████████████████████████████████████████▎                 | 5/6 [00:07<00:01,  1.46s/it]

[{'LinkSetDb': [{'Link': [{'Id': '2055019013'}, {'Id': '2055019012'}, {'Id': '2055019011'}, {'Id': '2055019010'}, {'Id': '2055019009'}, {'Id': '2055019008'}, {'Id': '2055019007'}, {'Id': '2055019006'}, {'Id': '2055019005'}, {'Id': '2055019004'}, {'Id': '2055019003'}, {'Id': '2055019002'}, {'Id': '2055019001'}, {'Id': '2055019000'}, {'Id': '2055018999'}, {'Id': '2055018998'}, {'Id': '2055018997'}, {'Id': '2055018996'}, {'Id': '2055018995'}, {'Id': '2055018994'}, {'Id': '2055018993'}, {'Id': '2055018992'}, {'Id': '2055018991'}, {'Id': '2055018990'}, {'Id': '2055018989'}, {'Id': '2055018988'}, {'Id': '2055018987'}, {'Id': '2055018986'}, {'Id': '2055018985'}, {'Id': '2055018984'}, {'Id': '2055018983'}, {'Id': '2055018982'}, {'Id': '2055018981'}, {'Id': '2055018980'}, {'Id': '2055018979'}, {'Id': '2055018978'}, {'Id': '2055018977'}, {'Id': '2055018976'}, {'Id': '2055018975'}, {'Id': '2055018974'}, {'Id': '2055018973'}, {'Id': '2055018972'}, {'Id': '2055018971'}, {'Id': '2055018970'}, {'Id':

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:08<00:00,  1.46s/it]

[{'LinkSetDb': [{'Link': [{'Id': '1937398188'}, {'Id': '1937398187'}, {'Id': '1937398185'}, {'Id': '1937398184'}, {'Id': '1937398183'}, {'Id': '1937398182'}, {'Id': '1937398181'}, {'Id': '1937398180'}, {'Id': '1937398178'}, {'Id': '1937398177'}, {'Id': '1937398176'}, {'Id': '1937398175'}, {'Id': '1937398174'}, {'Id': '1937398173'}, {'Id': '1937398171'}, {'Id': '1937398170'}, {'Id': '1937398169'}, {'Id': '1937398168'}, {'Id': '1937398167'}, {'Id': '1937398165'}, {'Id': '1937398164'}, {'Id': '1937398163'}, {'Id': '1937398162'}, {'Id': '1937398160'}, {'Id': '1937398159'}, {'Id': '1937398158'}, {'Id': '1937398157'}, {'Id': '1937398155'}, {'Id': '1937398153'}, {'Id': '1937398151'}, {'Id': '1937398150'}, {'Id': '1937398148'}, {'Id': '1937398146'}, {'Id': '1937398144'}, {'Id': '1937398142'}, {'Id': '1937398141'}, {'Id': '1937398139'}, {'Id': '1937398138'}, {'Id': '1937398136'}, {'Id': '1937398134'}, {'Id': '1937398132'}, {'Id': '1937398115'}, {'Id': '1937398094'}, {'Id': '1935596778'}], 'DbTo

In [61]:
# Collecting data about assemblies
gb_records = []
for link in tqdm(links):
    gb_handle = Entrez.efetch(db=db_current, rettype="gb", retmode="text", id=link[0])
    gb_record = SeqIO.read(gb_handle, 'genbank')
    gb_records.append((gb_record, link[1]))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 324/324 [09:34<00:00,  1.77s/it]


On the next stage we need to create fasta files for each assembly, to reaanotate them.

In [1]:
orglist = organism.split()
if len(orglist[-1]) < 10:
    last_letter = len(orglist[-1])
else:
    last_letter = 9
name = f"{orglist[0][0]}_{orglist[-1][:last_letter]}"
print(name)

P_penneri


In [23]:
dna_type, tuples, source_list = [], [], [] # Creating list for identyfing the number of every assemblie DNA molecules (chromosome and any plasmids)
orglist = organism.split()
if len(orglist[-1]) < 10:
    last_letter = len(orglist[-1])
else:
    last_letter = 9
name = f"{orglist[0][0]}_{orglist[-1][:last_letter]}"
for rec in tqdm(gb_records):
    source = rec[1] # Number of assembly
    if "plasmid" in rec[0].description:
        dna_type.append("plasmid")
    else:
        dna_type.append("chromosome")
    source_list.append(source) 
    number = source_list.count(source) # Counting the DNA molecule of assembly
    tuples.append((source, number))
    mask = f"{name}{source}_{number}"
    with open (f"../{name}/data/for_prokka_fasta/{mask}.fasta", "w") as for_prokka_fasta:
        for_prokka_fasta.write(">")
        for_prokka_fasta.write(mask)
        for_prokka_fasta.write("\n")
        for_prokka_fasta.write(str(rec[0].seq))
        for_prokka_fasta.write("\n")  

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 47/47 [00:00<00:00, 633.51it/s]


In [35]:
# Saving plasmid_code
jsonpc1 = json.dumps(tuples)
with open(f"../{name}/data/{name}_plasmid_code1.json", "w") as jspc1:
    jspc1.write(jsonpc1)
jsonpc2 = json.dumps(dna_type)
with open(f"../{name}/data/{name}_plasmid_code2.json", "w") as jspc2:
    jspc2.write(jsonpc2)